In [ ]:
from datasets import load_dataset,Dataset
START_SYM="<--start-->"
# use name="sample-10BT" to use the 10BT sample
# fw = load_dataset("/data/bert-chunker-v2/dataset/fineweb", name="sample/10BT", split="train", streaming=True)
# data_files = ["000_00000.parquet","001_00000.parquet","002_00000.parquet",]
data_files = ["para4000_cut.parquet",]

# fw = load_dataset("/data/bert-chunker-v2/dataset/fineweb/sample/10BT", data_files=data_files,split="train",num_proc=6,cache_dir='/data/bert-chunker-v2/dataset/cache')
# fw = load_dataset("/data/bert-chunker-v2/dataset/fineweb/sample/10BT",split="train",num_proc=6,cache_dir='/data/bert-chunker-v2/dataset/cache')
fw = load_dataset("/data/bert-chunker-v2/dataset/doubao_responses", data_files=data_files,split="train",num_proc=6,cache_dir='/data/bert-chunker-v2/dataset/cache')

print(fw)


In [ ]:

print(fw[1]['chunk'])



In [ ]:
d = fw.filter(lambda x: 'microcomputer' in x['input'],num_proc=6)
print(d)
print(d[2]['input'])

In [ ]:
# d = fw.filter(lambda x: x["text"].count('\n')>=2 and x['token_count']<=150,num_proc=6)
# d = fw.filter(lambda x:  x['token_count']<=150 and x['token_count']>=50,num_proc=6)
d = fw.filter(lambda x:  x['token_count']>250,num_proc=6)

# def cut(example):
#     example["input"] = example['text']
#     return example

# # 使用 map 方法批量处理
# d = d.map(cut,num_proc=6)
# last_100_elements = d[0:10000]

# # 形成新的数据集
# d = Dataset.from_dict(last_100_elements)
# d.to_parquet("/data/bert-chunker-v2/dataset/fw-train-onepara.parquet")


In [ ]:
print(d)

In [ ]:
print(d[2]['text'])

print(d[2]['token_count'])
d = d.shuffle(seed=42)
print(d[2]['text'])

print(d[2]['token_count'])

In [ ]:
print(d)

In [ ]:
def cut(example):
    START_SYM="<--start-->"
    example["input"] = example['text']+START_SYM
    return example

# 使用 map 方法批量处理
d = d.map(cut,num_proc=6)


In [19]:
d = d.remove_columns(['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'token_count'])

In [ ]:
def merge_batch(batch, group_size=3):
    START_SYM="<--start-->"
    new_data = {"input": [],}
    for i in range(0, len(batch['input']), group_size):
        # 获取当前组的 4 条数据
        texts = batch['input'][i:i+group_size]
        
        # 合并数据
        merged_text = ' '.join(texts)
        merged_text = merged_text[:-len(START_SYM)]
        
        # 添加到新数据中
        new_data["input"].append(merged_text)
    return new_data

# 使用 Dataset.map 处理数据
new_dataset = d.map(
    merge_batch,
    batched=True,  # 启用批处理
    batch_size=1000,  # 每批次处理 1000 条原始数据
    remove_columns=d.column_names,  # 移除原始列
    num_proc=6,  # 使用 6 个进程并行处理
)

In [ ]:
print(new_dataset)

In [ ]:
# 获取最后 100 个元素
last_100_elements = new_dataset[-10000:]

# 形成新的数据集
d = Dataset.from_dict(last_100_elements)
print(d)

In [ ]:
d.to_parquet("/data/bert-chunker-v2/dataset/fw-test-cross.parquet")